# Utility to manage the Actuator Disk runs
## Prerequisite setup

Create a project with the following folder structure
```
windsim-projects
-my_AD_project
--my_AD_project_base
```

Set up the base project by the following steps:
1. Import grid file (.gws)
2. Run Terrain (Resolution doesn't matter, set wanted extent)
3. Set Windfields settings
4. Go to objects park layout and set turbines and climatologies
5. Tools-->export objects ('.ows')

* Alternatively you have you need a project and an object file

# NOTE to self
Remember to clear folder before, not sure if it overwrites or deletes files. Can cause issues


In [9]:
# Python needs to be >= 3.9
from pathlib import Path
import sys
if sys.version_info < (3,8):
    print(f"Python version is {sys.version_info}, needs to be 3.9 or newer.")


## Project and environment variables
### System
* windsim: WindSim installation folder
* environment: Where your environment file is located
* layout: Name of the layout with suffix
### Project 
* project: Directory where your project lives
* owsfile: Name of the layout, containing the turbines you want to use as Actuator disks
* layout: Name of the layout with suffix
* low_windspeed: What windspeed should be used for the low windspeed
* high_windspeed: What windspeed should be used for the high windspeed
* AD_spacings: What is the number of cells used for the AD, 16 is the recommended. This influences the size of the resulting projects.

In [11]:
# Defining what you want to run
project = Path('C:/Users/GullikKillie/Documents/WindSim Projects 12/Actuator_Disk_Flat/Actuator_Disk_Flat/Actuator_Disk_Flat.ws')
windsim=Path('C:/Program Files/WindSim/WindSim 12.0.0')
environment=Path('C:/Users/GullikKillie/AppData/Roaming/WindSim/1200/Environment.xml')
owsfile='AD_layout'
layout='Layout 1.lws'

# AD specific settings
low_windspeed=7
high_windspeed=20
AD_spacings=2

## Project
Defining the project class, project specific information

In [12]:
# Defining the project we want to work with
class Project():
    def __init__(self, project_path: Path, layout: str="Layout 1.lws"):
        self.name = project_path.stem
        self.project_file = project_path
        self.layout = layout
        self.base_directory = self.project_file.parent.parent


## AD runner
ActuatorDiskRunner is the runner responsible for setting up and running the project.
It is responsible to copy the base project, as well as managing some configuration files.

In [13]:
# Defining the Actuator disk class
from pathlib import Path    
import shutil
import xml.etree.ElementTree as ET
from typing import List
import itertools
from pprint import pprint
import os
import time
import types
import subprocess




# Defining the Actuator disk class
class ActuatorDiskRunner:
    def __init__(self, project_path: Path=Path('C:/Users/GullikKillie/Documents/WindSim Projects 12/Actuator_Disk_Flat/Actuator_Disk_Flat_base/Actuator_Disk_Flat.ws'), windsim=Path('C:/Program Files/WindSim/WindSim 12.0.0'), environment=Path('C:/Users/GullikKillie/AppData/Roaming/WindSim/1200/Environment.xml'), owsfile: str='AD_layout', layout='Layout 1.lws'):
        self.project = Project(project_path=project_path, layout=layout)
        self.owsfile = owsfile
        self.project_names = []
        self.windsim = windsim
        self.environment = environment
        self.sectors = self._get_sectors()
        self.threads = self._get_threads()

    def setup_AD(self, windspeeds: List[float] = [7, 20], AD_spacings: int=16):
        """ Creates projects and necessary files to run the actuator disks"""
        
        for AD, windspeed in itertools.product([True, False], windspeeds):
            print(f'Setting up {self.project.name} with AD: {AD} and Windspeed {windspeed}')
            copy_name = self.copy_project(AD=AD, windspeed=windspeed, AD_spacings=AD_spacings)
            self.project_names.append(copy_name)

        print(f'Adding the {self.project.base_directory}/<project>/<layout>/log/blockage_effect.log file to all projects')
        for project_name in self.project_names:
            with open(self.project.base_directory / project_name / self.project.layout.strip('.lws') / 'log' / 'blockage_effect.log', 'w') as outfile:
                outfile.write('\n'.join(str(self.project.base_directory / f'{i}') + '\\' for i in self.project_names))

        return
    

    def _get_sectors(self) -> List[float]:
        """ Read the project file and the number of sectors to run for the projects """
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(self.project.project_file)
        root = tree.getroot()
        sectors = []
        for sector_element in root.iter('{ProjectParameters.xsd}Sector'):
            sectors.append(sector_element.text)

        return sectors
    
    def _get_threads(self) -> int:
        """ Read the project file and the number of simultaneous simulations to run for the projects """
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(self.project.project_file)
        root = tree.getroot()
        threads = 1
        for threads_element in root.iter('{ProjectParameters.xsd}ParallelCores'):
            threads = threads_element.text

        return threads

    
    def _replace_or_add_element(self, namespace, field: ET.Element, parameter: str, value: any):
        # This may or may not be available
        parameter_present = False
        for child in field.findall("{ProjectParameters.xsd}"+parameter, namespace):
            parameter_present = True
            child.text = str(value)
        if not parameter_present:
            ows_element = ET.SubElement(field, parameter)
            ows_element.text = str(value)

    def copy_project(self, AD: bool = True, windspeed: float = 7, AD_spacings: int = 8) -> str: 
        """Copies a project with different settings given by the input"""

        copy_name = f'{self.project.name}_AD_{AD}_windspeed_{windspeed}'
        print(f"Making {copy_name} from base {self.project.name}")
        base_dir = self.project.project_file.parent
        dest_dir = self.project.base_directory / copy_name
        
        shutil.copytree(base_dir, dest_dir, dirs_exist_ok=True)

        #Changing the Number of nodes and cells in the project
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(dest_dir  / f"{self.project.name}.ws")
        root = tree.getroot()

        namespace = {'project_parameters': 'ProjectParameters.xsd'}

        for WindFields in root.findall("{ProjectParameters.xsd}WindField"):
            self._replace_or_add_element(namespace, WindFields, "VelocityBoundaryLayer", windspeed)

        if AD:
            for CFD in root.findall("{ProjectParameters.xsd}CFD"):
                self._replace_or_add_element(namespace, CFD, "RefinementType", 3)
                self._replace_or_add_element(namespace, CFD, "NumberSpacings", AD_spacings)
                self._replace_or_add_element(namespace, CFD, "OWSFile", self.owsfile)

        else:
            for CFD in root.findall("{ProjectParameters.xsd}CFD"):
                self._replace_or_add_element(namespace, CFD, "RefinementType", 2)
                self._replace_or_add_element(namespace, CFD, "RefinementFileName", 'actuator_discs.bws')
        

        ET.indent(tree, '  ')
        tree.write(dest_dir  / f"{self.project.name}.ws", encoding = "UTF-8",  xml_declaration=True)

        project_file = dest_dir  / f"{self.project.name}.ws"
        project_file.replace(f'{dest_dir / copy_name}.ws')

        return copy_name


### Run Setup project
The python program will then create 4 copies of the base project so the directory will look like:
```
windsim-projects
-my_AD_project
--my_AD_project
--my_AD_project_AD_True_windspeed_7
--my_AD_project_AD_True_windspeed_20
--my_AD_project_AD_False_windspeed_7
--my_AD_project_AD_False_windspeed_20
```
Setting up a project for each of the combinations of windspeed: [low, high] and ActuatorDisk [with, without]

In [14]:
# Running the setup script
from pathlib import Path
runner = ActuatorDiskRunner(project_path=project, windsim=windsim, environment=environment, owsfile=owsfile, layout=layout)

runner.setup_AD(windspeeds=[low_windspeed, high_windspeed], AD_spacings=AD_spacings)  # Copy projects and edit project.ws file


Setting up Actuator_Disk_Flat with AD: True and Windspeed 7
Making Actuator_Disk_Flat_AD_True_windspeed_7 from base Actuator_Disk_Flat
Setting up Actuator_Disk_Flat with AD: True and Windspeed 20
Making Actuator_Disk_Flat_AD_True_windspeed_20 from base Actuator_Disk_Flat
Setting up Actuator_Disk_Flat with AD: False and Windspeed 7
Making Actuator_Disk_Flat_AD_False_windspeed_7 from base Actuator_Disk_Flat
Setting up Actuator_Disk_Flat with AD: False and Windspeed 20
Making Actuator_Disk_Flat_AD_False_windspeed_20 from base Actuator_Disk_Flat
Adding the C:\Users\GullikKillie\Documents\WindSim Projects 12\Actuator_Disk_Flat/<project>/<layout>/log/blockage_effect.log file to all projects


## Run Terrain
This runs the Terrain for each of the created projects.
The Actuator Disk project needs to run first, since they create the actuator_disk.bws file, since it sets the grid the free windspeed simulations also need to use.


In [15]:

def run_Terrains(self):
    # First run the AD projects
    for project_name in self.project_names:
        if 'AD_True' in project_name:

            self.run_Terrain(project_name)
            shutil.copy2(self.project.base_directory / project_name / 'dtm' / 'actuator_discs.bws', self.project.base_directory)
    # Copy actuator_discs.bws to all files
    for project_name in self.project_names:
        shutil.copy2(self.project.base_directory /'actuator_discs.bws', self.project.base_directory / project_name / 'dtm')
    for project_name in self.project_names:
        if 'AD_False' in project_name:
            self.run_Terrain(project_name)

    print("Finished running Terrain")
    

def run_Terrain(self, name: str):
    os.makedirs('tmp/', exist_ok=True)
    os.chdir('tmp')
    
    print(f'Run Terrain for {name}')
    subprocess.run(f'"{self.windsim / "bin" / "Terrain.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}"')
        
    print(f'Run Report[Terrain] for {name}')

    subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 1')

    os.chdir('../')

    shutil.rmtree('tmp')


# Add Terrain runner to runner
runner.run_Terrains = types.MethodType(run_Terrains, runner)
runner.run_Terrain = types.MethodType(run_Terrain, runner)

runner.run_Terrains()


Run Terrain for Actuator_Disk_Flat_AD_True_windspeed_7
Run Report[Terrain] for Actuator_Disk_Flat_AD_True_windspeed_7
Run Terrain for Actuator_Disk_Flat_AD_True_windspeed_20
Run Report[Terrain] for Actuator_Disk_Flat_AD_True_windspeed_20
Run Terrain for Actuator_Disk_Flat_AD_False_windspeed_7
Run Report[Terrain] for Actuator_Disk_Flat_AD_False_windspeed_7
Run Terrain for Actuator_Disk_Flat_AD_False_windspeed_20
Run Report[Terrain] for Actuator_Disk_Flat_AD_False_windspeed_20
Finished running Terrain


## Run WindFields
This runs WindFields for each of the projects. First we make a simple version for simpleness, and then we will expand it to a run all sectors in parallel.
This version is only running the single sector of each project.

In [7]:
def run_WindFields(self):
    # First run the AD projects
    for project_name in self.project_names:
        if 'AD_True' in project_name:
            self.run_WindField(project_name)

    # Then run the non-AD projects
    for project_name in self.project_names:
        if 'AD_False' in project_name:
            self.run_WindField(project_name)

    print("Finished running WindFields")

def run_WindField(self, name: str):
    os.makedirs('tmp/', exist_ok=True)
    os.chdir('tmp')

    print(f'Running WindFields for {name}, sector: {self.sectors[0]}')
    command = f'"{self.windsim / "bin" / "WindFields.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" /si1'
    p=subprocess.run(command)            
    
    print(f'Running Report[WindFields] for {name}')
    subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 2')

    os.chdir('../')

    shutil.rmtree('tmp')


# Add Terrain runner to runner
runner.run_WindFields = types.MethodType(run_WindFields, runner)
runner.run_WindField = types.MethodType(run_WindField, runner)

runner.run_WindFields()



Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 0
Running Report[WindFields] for Actuator_Disk_Flat_AD_True_windspeed_7
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_20, sector: 0
Running Report[WindFields] for Actuator_Disk_Flat_AD_True_windspeed_20
Running WindFields for Actuator_Disk_Flat_AD_False_windspeed_7, sector: 0
Running Report[WindFields] for Actuator_Disk_Flat_AD_False_windspeed_7
Running WindFields for Actuator_Disk_Flat_AD_False_windspeed_20, sector: 0
Running Report[WindFields] for Actuator_Disk_Flat_AD_False_windspeed_20
Finished running WindFields


### The final Windfields
Now since in a proper project we want to run all the sectors for all the projects and that may take some time. So we want to be able to run them in parallel, using the setting from the WindSim file. Then we overwrite the run_WindFields method with the new version.

In [8]:

def run_WindField(self, name: str):
    """Remember to add sleep when all prescribed threads are used"""
    os.makedirs('tmp/', exist_ok=True)
    os.chdir('tmp')
    i = 1
    processes = []
    for sector in self.sectors:

        print(f'Running WindFields for {name}, sector: {sector}')
        command = f'"{self.windsim / "bin" / "WindFields.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" /si{i}'
        p=subprocess.Popen(command,stdout=subprocess.PIPE,stderr=subprocess.PIPE)            
        processes.append(p)
        time.sleep(10)
        i += 1
    i = 0
    for p in processes:
        stdout,stderr = p.communicate()
        print(f"Windfield Out Sector {self.sectors[i]}: ",stdout)
        i+=1
    i=0
    for cp in processes:
        cp.wait()
        stdout,stderr = p.communicate()
        if p.returncode != 0:
            print(f"Windfield Error Sector {self.sectors[i]}: ",stderr)        
        i+=1
    
    print(f'Running Report[WindFields] for {name}')
    subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 2')

    os.chdir('../')

    shutil.rmtree('tmp')


# Add Terrain runner to runner
runner.run_WindField = types.MethodType(run_WindField, runner)

runner.run_WindFields()


Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 0


Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 30
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 60
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 90
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 120
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 150
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 180
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 210
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 240
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 270
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 300
Running WindFields for Actuator_Disk_Flat_AD_True_windspeed_7, sector: 330
Windfield Out Sector 0:  b'\r\n   WARNING .................\r\n\r\n   in: read_xml_ProjectParameters\r\n\r\nUsing a number of spacings smaller then 16 introduces significant disc

# Running it outside the notebook

There is also an attached version of the AD-runner in the directory.

To use it make the prep project and run, from your python environment:

```sh
python actuator_disk.py --mode 'all' --project 'C:/Users/<User>/Documents/WindSim Projects 12/Actuator_Disk_Flat/Actuator_Disk_Flat/Actuator_Disk_Flat.ws' --windsim 'C:/Program Files/WindSim/WindSim 12.0.0' --environment 'C:/Users/<User>/AppData/Roaming/WindSim/1200/Environment.xml' --owsfile 'AD_layout' --layout 'Layout 1.lws
```

Use '--mode setup' to do only the setup and so on, '--mode terrain' to run untill terrain and so on. User '--help' for an overview of usage.

```sh
python actuator_disk.py --help
```



## The full class

In [ ]:
# Defining the Actuator disk class
from pathlib import Path    
import shutil
import xml.etree.ElementTree as ET
from typing import List
import itertools
from pprint import pprint
import os
import subprocess



# Defining the project we want to work with
class Project():
    def __init__(self, project_path: Path, layout: str="Layout 1.lws"):
        self.name = project_path.stem
        self.project_file = project_path
        self.layout = layout
        self.base_directory = self.project_file.parent.parent

class ActuatorDiskRunner:
    def __init__(self, project_path: Path=Path('C:/Users/GullikKillie/Documents/WindSim Projects 12/Actuator_Disk_Flat/Actuator_Disk_Flat_base/Actuator_Disk_Flat.ws'), windsim=Path('C:/Program Files/WindSim/WindSim 12.0.0'), environment=Path('C:/Users/GullikKillie/AppData/Roaming/WindSim/1200/Environment.xml'), owsfile: str='AD_layout', layout='Layout 1.lws'):
        self.project = Project(project_path=project_path, layout=layout)
        self.owsfile = owsfile
        self.project_names = []
        self.windsim = windsim
        self.environment = environment

    def setup_AD(self, windspeeds: List[int] = [7, 20]):
        """ Creates projects and necessary files to run the actuator disks"""
        
        for AD, windspeed in itertools.product([True, False], windspeeds):
            print(f'Setting up {self.project.name} with AD: {AD} and Windspeed {windspeed}')
            copy_name = self.copy_project(AD=AD, windspeed=windspeed)
            self.project_names.append(copy_name)

        print(f'Adding the {self.project.base_directory}/<project>/<layout>/log/blockage_effect.log file to all projects')
        for project_name in self.project_names:
            with open(self.project.base_directory / project_name / self.project.layout.strip('.lws') / 'log' / 'blockage_effect.log', 'w') as outfile:
                outfile.write('\n'.join(str(self.project.base_directory / i) + '\\' for i in self.project_names))

        return
    
    def _replace_or_add_element(self, namespace, field: ET.Element, parameter: str, value: any):
        # This may or may not be available
        parameter_present = False
        for child in field.findall("{ProjectParameters.xsd}"+parameter, namespace):
            parameter_present = True
            child.text = str(value)
        if not parameter_present:
            ows_element = ET.SubElement(field, parameter)
            ows_element.text = str(value)

    def copy_project(self, AD: bool = True, windspeed: int = 7) -> str: 

        """Copies a project with different settings given by the input"""
        copy_name = f'{self.project.name}_AD_{AD}_windspeed_{windspeed}'
        print(f"Making {copy_name} from base {self.project.name}")
        base_dir = self.project.project_file.parent
        dest_dir = self.project.base_directory / copy_name
        
        shutil.copytree(base_dir, dest_dir, dirs_exist_ok=True)

        #Changing the Number of nodes and cells in the project
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(dest_dir  / f"{self.project.name}.ws")
        root = tree.getroot()

        namespace = {'project_parameters': 'ProjectParameters.xsd'}

        for WindFields in root.findall("{ProjectParameters.xsd}WindField"):
            self._replace_or_add_element(namespace, WindFields, "VelocityBoundaryLayer", windspeed)

        if AD:
            for CFD in root.findall("{ProjectParameters.xsd}CFD"):
                self._replace_or_add_element(namespace, CFD, "RefinementType", 3)
                self._replace_or_add_element(namespace, CFD, "NumberSpacings", 1)
                self._replace_or_add_element(namespace, CFD, "OWSFile", self.owsfile)

        else:
            for CFD in root.findall("{ProjectParameters.xsd}CFD"):
                self._replace_or_add_element(namespace, CFD, "RefinementType", 2)
                self._replace_or_add_element(namespace, CFD, "RefinementFileName", 'actuator_discs.bws')
        

        ET.indent(tree, '  ')
        tree.write(dest_dir  / f"{self.project.name}.ws", encoding = "UTF-8",  xml_declaration=True)

        project_file = dest_dir  / f"{self.project.name}.ws"
        project_file.replace(f'{dest_dir / copy_name}.ws')

        return copy_name
        
    def run_Terrains(self):
        # First run the AD projects
        for project_name in self.project_names:
            if 'AD_True' in project_name:
    
                self.run_Terrain(project_name)
                shutil.copy2(self.project.base_directory / project_name / 'dtm' / 'actuator_discs.bws', self.project.base_directory)
        # Copy actuator_discs.bws to all files
        for project_name in self.project_names:
            shutil.copy2(self.project.base_directory /'actuator_discs.bws', self.project.base_directory / project_name / 'dtm')
        for project_name in self.project_names:
            if 'AD_False' in project_name:
                self.run_Terrain(project_name)
        
    
    def run_Terrain(self, name: str):
        os.makedirs('tmp/', exist_ok=True)
        os.chdir('tmp')
        
        print(f'Run Terrain for {name}')
        subprocess.run(f'"{self.windsim / "bin" / "Terrain.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}"')
            
        print(f'Run Report[Terrain] for {name}')
    
        subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 1')
    
        os.chdir('../')
    
        shutil.rmtree('tmp')

    def run_WindFields(self):
        # First run the AD projects
        for project_name in self.project_names:
            if 'AD_True' in project_name:
                self.run_WindField(project_name)
        # Then run the non-AD projects
        for project_name in self.project_names:
            if 'AD_False' in project_name:
                self.run_WindField(project_name)
        
    
    def run_WindField(self, name: str):
        os.makedirs('tmp/', exist_ok=True)
        os.chdir('tmp')
    
        print(f'Running WindFields for {name}')
        subprocess.run(f'"{self.windsim / "bin" / "WindFields.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" /si1')
        
        print(f'Running Report[WindFields] for {name}')
        subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 2')
    
        os.chdir('../')
    
        shutil.rmtree('tmp')

# Running the setup script
from pathlib import Path
runner = ActuatorDiskRunner(project_path=project, windsim=windsim, environment=environment, owsfile=owsfile, layout=layout)

runner.setup_AD()  # Copy projects and edit project.ws file
runner.run_Terrains()
runner.run_WindFields()


